In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import * 
lap_times_schema = StructType([StructField('raceId', IntegerType(), True),
                               StructField('driverId', IntegerType(), True),
                               StructField('lap', IntegerType(), True),
                               StructField('position', IntegerType(), True),
                               StructField('time', StringType(), True),
                               StructField('milliseconds', IntegerType(), True)])

In [0]:
lap_times_df = spark.read.schema(lap_times_schema).csv(f"{raw_container_folder_path}/{v_file_date}/lap_times")

In [0]:
from pyspark.sql.functions import current_timestamp,lit
lap_times_final_df = lap_times_df.withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("driverId", "driver_id")\
    .withColumn("ingested_date", current_timestamp())\
    .withColumn("data_source", lit(v_data_source))

In [0]:
#lap_times_final_df.write.mode("overwrite").parquet(f"{processed_container_folder_path}/lap_times")

# lap_times_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed_db.lap_times")

#load_incremental_data(lap_times_final_df, "race_id", "f1_processed_db", "lap_times")

merge_condition = "target_tab.driver_id = source_tab.driver_id AND target_tab.race_id = source_tab.race_id AND target_tab.lap = source_tab.lap AND target_tab.race_id = source_tab.race_id"
merge_incremental_data( lap_times_final_df, processed_container_folder_path, merge_condition, "race_id", "f1_processed_db", "lap_times")

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
select distinct(race_id), count(*) from f1_processed_db.lap_times group by race_id order by race_id desc